# Analysis of Stock Market Investment Strategies

Author: Charles Lee

This Notebook aims to compare three different investment strategies using historical stock market data.
1. Timing the Market (Perfect) - Buying shares when shares are lowest every year
2. Take Action (Now) - Buying shares with all their money at the beginning of every year
3. Dollar Cost Averaging (Average) - Dividing up the money to buy shares at the beginning of every month

We will also take a look at a "worst-case scenario"

4. Unlucky (Worst) - Saved up all their money and bought all right before the 2008 Stock Market Crash and 2020 COVID-19 Pandemic

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import datetime
import calendar

## Case Study of VTI
VTI is a highly popular ETF that holds shares from the top companies in America

In [ ]:
vti = pd.read_csv('VTI.csv')
vti

In [ ]:
vti_div = pd.read_csv('VTI_Dividends.csv')
vti_div

In [ ]:
standardize_date = lambda string: datetime.datetime.strptime(string, '%Y-%m-%d')
vti['Date'] = vti['Date'].apply(standardize_date)
vti_div['Date'] = vti_div['Date'].apply(standardize_date)

### Here is a graph of VTI's stock price per day since 2001

In [ ]:
plt.figure(figsize=(16, 6))
g = sns.lineplot(x='Date', y='High', data=vti, ci=None)
plt.xticks([datetime.datetime(year=x, month=1, day=1) for x in range(2001, 2023)], rotation=45)
g.set_xticklabels(list(range(2001, 2023)))
g.set_xlabel('Year')
g.set_ylabel('Stock Price')
g.set_title('VTI High Stock Price Per Day')
sns.despine()

In [ ]:
class Investor():
    def __init__(self, stock):
        # Creates portfolio and binds a stock
        self.portfolio = pd.DataFrame(columns=['Date', 'Purchase Price'])
        #self.portfolio_low = pd.DataFrame(columns=['Date', 'Purchase Price'])
        self.salary = 6000
        self.cash = 0
        self.stock = stock
        self.profits = pd.DataFrame(columns=['Date', 'Profits'])
        self.first_date = self.stock['Date'][0]
        self.last_date = self.stock['Date'][len(vti)-1]
        
    def income(self):
        # Yearly disbursement of money
        self.cash += self.salary
    
    def create_date(self, string):
        # Transforms date string to datetime object
        if isinstance(string, str):
            return standardize_date(string)
        else:
            return string
    
    def get_nearest_date(self, date, reverse = False):
        # Gets the next closest date of the stock market being open and returns the row of data
        date = self.create_date(date)
        if reverse:
            move = -1
        else:
            move = 1
        # Changes date until found data, unless past first or last date
        while len(self.stock[self.stock['Date'] == date]) < 1:
            if (move == 1 and date > self.last_date):
                # Later than last, return last
                date = self.stock['Date'][len(self.stock)-1]
                row = self.stock[self.stock['Date'] == date]
                return row, date
            elif (move == -1 and date < self.first_date):
                # Earlier than first, return first
                date = self.stock['Date'][0]
                row = self.stock[self.stock['Date'] == date]
                return row, date
            date += datetime.timedelta(days=move)
        # Gets row data from found date
        row = self.stock[self.stock['Date'] == date]
        return row, date
    
    def get_stock_price(self, date):
        date = self.create_date(date)
        row, date = self.get_nearest_date(date)
        return row['High'].values[0]
    
    def buy_stock(self, date):
        # Buys a stock and return True if successful and False if not
        date = self.create_date(date)
        row, date = self.get_nearest_date(date)
        
        # Get stock price
        price = row['High'].values[0]
        
        # Buy if can afford
        if self.cash >= price:
            self.portfolio = self.portfolio.append(pd.DataFrame([[date, price]], columns=['Date', 'Purchase Price'])).reset_index(drop=True)
            self.cash -= price
            return True
        return False
    
    def get_invested(self, date=None):
        # Returns how much money you've put into the stock market
        if date is not None:
            date = self.create_date(date)
            row, date = self.get_nearest_date(date)
            partial = self.portfolio[self.portfolio['Date'] <= date]
            return partial.sum()[0]
        return self.portfolio.sum()[0]
    
    def get_value(self, date=None):
        # Returns how much the stocks are worth at a given date
        if date is not None:
            date = self.create_date(date)
            row, date = self.get_nearest_date(date)
            price = row['High'].values[0]
            partial = self.portfolio[self.portfolio['Date'] <= date]
            return len(partial) * price
        price = self.stock['High'][len(self.stock)-1]
        return len(self.portfolio) * price
    
    def execute_strategy(self):
        pass
    
    def find_profits(self, year):
        for i in range(1, 13):
            date = datetime.datetime(year=year, month=i, day=calendar.monthrange(year, i)[1])
            row, date = self.get_nearest_date(date, reverse=True)
            profit = self.get_value(date) - self.get_invested(date)
            self.profits = self.profits.append(pd.DataFrame([[date, profit]], columns=['Date', 'Profits']))
        self.profits = self.profits.drop_duplicates().reset_index(drop=True)
    
class Perfect(Investor):
    # Buy at the lowest point of every month/year?
    pass

class Now(Investor):
    # Use all money to buy shares at beginning of year
    def execute_strategy(self, start, end):
        for i in range(start, end+1):
            self.income()
            date = datetime.datetime(year=i, month=1, day=1)
            can_buy = True
            while can_buy:
                can_buy = self.buy_stock(date)
            self.find_profits(i)

class Average(Investor):
    # Portion out money over year and buy shares at beginning of each month
    def execute_strategy(self, start, end):
        budget = 0
        for i in range(start, end+1):
            # Every year
            self.income()
            for j in range(1, 13):
                # Every month
                budget += self.salary/12
                date = datetime.datetime(year=i, month=j, day=1)
                try:
                    price = self.get_stock_price(date)
                except:
                    break
                while budget >= price:
                    bought = self.buy_stock(date)
                    if bought:
                        budget -= price
            self.find_profits(i)
                    


## Take Action (Now) Type Investor
This investor uses all of their money to buy shares at beginning of year

In [ ]:
now = Now(vti)
now.execute_strategy(2001, 2021)

This is the leftover uninvested cash by the end of strategy execution

In [ ]:
now.cash

This is the investor's portfolio by the end of the strategy execution

In [ ]:
now.portfolio

This is how much money the investor spent on buying the stocks

In [ ]:
now.get_invested()

This is how much money the investments are worth if we are to sell at the given date

In [ ]:
date = '2021-1-04'
now.get_value(date)

In [ ]:
now.profits

In [ ]:
plt.figure(figsize=(16, 6))
g = sns.lineplot(x='Date', y='Profits', data=now.profits)
plt.xticks([datetime.datetime(year=x, month=1, day=1) for x in range(2001, 2023)], rotation=45)
g.set_xticklabels(list(range(2001, 2023)))
g.set_xlabel('Year')
g.set_ylabel('Profit')
g.set_title('Profits Made Per Month From Investments Using Now strategy')
sns.despine()

## Dollar Cost Averaging (Average) Type Investor
This investor portions out their money over year and buys shares at beginning of each month (Dollar Cost Averaging)

In [ ]:
avg = Average(vti)
avg.execute_strategy(2001, 2021)

In [ ]:
# The leftover uninvested cash by the end of strategy execution
avg.cash

This is the investor's portfolio by the end of the strategy execution

In [ ]:
avg.portfolio

This is how much money the investor spent on buying the stocks

In [ ]:
avg.get_invested()

This is how much money the investments are worth if we are to sell at the given date

In [ ]:
date = '2021-1-04'
avg.get_value(date)

In [ ]:
avg.profits

In [ ]:
plt.figure(figsize=(16, 6))
g = sns.lineplot(x='Date', y='Profits', data=avg.profits)
plt.xticks([datetime.datetime(year=x, month=1, day=1) for x in range(2001, 2023)], rotation=45)
g.set_xticklabels(list(range(2001, 2023)))
g.set_xlabel('Year')
g.set_ylabel('Profit')
g.set_title('Profits Made Per Month From Investments')
sns.despine()

## Putting it all together

In [ ]:
g_now = now.profits
g_avg = avg.profits

g_now['Strategy'] = 'Now'
g_avg['Strategy'] = 'Average'

In [ ]:
g_all = g_now.append(g_avg)

In [ ]:
sns.lineplot(data=g_all, x='Date', y='Profits', hue='Strategy')

In [ ]:
date = g_all.sort_values('Date')['Date'][len(g_all)/2-1].values[0]
sns.catplot(x='Strategy', y='Profits', data=g_all[g_all['Date'] == date], kind='bar')

## Unit Tests

In [ ]:
test = Investor(vti)
assert test.cash == 0
test.income()
assert test.cash == test.salary

In [ ]:
assert test.create_date('2001-05-31') == datetime.datetime(year=2001, month=5, day=31)

In [ ]:
# Test before first forward
test_row, test_date = test.get_nearest_date('2001-06-13')
assert test_date == test.create_date('2001-06-15')
# Test after last forward
test_row, test_date = test.get_nearest_date('2021-02-17')
assert test_date == test.create_date('2021-02-16')
# Test before first reverse
test_row, test_date = test.get_nearest_date('2001-06-13', True)
assert test_date == test.create_date('2001-06-15')
# Test after last reverse
test_row, test_date = test.get_nearest_date('2021-02-17', True)
assert test_date == test.create_date('2021-02-16')
# Test forward
test_row, test_date = test.get_nearest_date('2021-02-13')
assert test_date == test.create_date('2021-02-16')
# Test reverse
test_row, test_date = test.get_nearest_date('2001-06-17', True)
assert test_date == test.create_date('2001-06-15')

In [ ]:
assert test.get_stock_price('2001-06-15') == 56.005001
assert test.get_stock_price('2001-06-14') == 56.005001
assert test.get_stock_price('2001-06-17') == 55.915001

In [ ]:
# Not enough money
test.cash = 0
assert not test.buy_stock('2001-06-15')
# Enough money
test.cash = 200
assert test.buy_stock('2001-06-15')

In [ ]:
test = Investor(vti)
test.cash = 9999999
for i in range(100):
    test.buy_stock('2001-06-15')
    test.buy_stock('2021-02-16')

# All
np.testing.assert_almost_equal(test.get_invested(), 56.005001*100 + 208.389999*100)
# Before specific date
np.testing.assert_almost_equal(test.get_invested('2002-01-01'), 56.005001*100)

In [ ]:
test = Investor(vti)
test.cash = 9999999
for i in range(100):
    test.buy_stock('2001-06-15')
    test.buy_stock('2021-02-16')

# All
np.testing.assert_almost_equal(test.get_value(), 200*208.389999)
# Before specific date
np.testing.assert_almost_equal(test.get_value('2001-06-21'), 56.724998*100)

In [ ]:
test = Investor(vti)
test.cash = 9999999
for i in range(100):
    test.buy_stock('2001-06-15')

test.find_profits(2001)
np.testing.assert_almost_equal(test.profits[test.profits['Date'] == '2001-06-15']['Profits'].values[0], 0)

np.testing.assert_almost_equal(test.profits[test.profits['Date'] == '2001-06-29']['Profits'].values[0], (56.81-56.005)*100)